In [53]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [54]:
arg_date = "2021-08-26"
src_format = "%Y-%m-%d"
src_bucket = "deutsche-boerse-xetra-pds"
trg_bucket = 'xetra-daily'
columns = ['ISIN','Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
key = "xetra_daily_report" + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [55]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [56]:
arg_date_dt

datetime.date(2021, 8, 25)

In [57]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], src_format).date() >= arg_date_dt]

In [58]:
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-08-25/2021-08-25_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [59]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get("Body").read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data,delimiter=",")
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index = True)

In [60]:
df_all = df_all.loc[:,columns]

In [61]:
df_all.dropna(inplace=True)
df_all.shape

(257879, 8)

## Get opening price per ISIN and day

In [62]:
df_all['opening_price'] = df_all.sort_values(by = "Time").groupby(['ISIN','Date'])['StartPrice'].transform('first')

## Closing Price per ISIN and day

In [63]:
df_all['closing_price'] = df_all.sort_values(by = "Time").groupby(['ISIN','Date'])['EndPrice'].transform('last')

## Aggregations

In [64]:
df_all = df_all.groupby(['ISIN','Date'], as_index = False).agg(opening_price_eur=('opening_price','min'),closing_price_eur = ('closing_price','min'),minimum_price_eur=('MinPrice','min'),maximum_price_eur=('MaxPrice','max'),daily_traded_volume=('TradedVolume','sum'))

## Percent Change Prev Closing

In [65]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [66]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [67]:
df_all.drop(columns = ['prev_closing_price'],inplace = True)

In [68]:
df_all = df_all.round(decimals = 2)


In [69]:
df_all = df_all[df_all.Date >= arg_date]

## Write to S3

In [70]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer,index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key = key)

s3.Object(bucket_name='xetra-daily', key='xetra_daily_report20210829_094217.parquet')

## Reading the uploaded file

In [71]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report20210829_080542.parquet
xetra_daily_report20210829_094217.parquet


In [74]:
prq_obj = bucket_target.Object(key="xetra_daily_report20210829_094217.parquet").get().get("Body").read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [75]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-08-26,8.86,8.59,8.59,8.86,1920,-2.61
1,AT00000FACC2,2021-08-27,8.65,8.63,8.51,8.65,613,0.47
2,AT0000606306,2021-08-26,20.58,20.42,20.38,20.58,828,-0.68
3,AT0000606306,2021-08-27,20.36,20.34,20.24,20.46,583,-0.39
4,AT0000609607,2021-08-26,15.90,16.28,15.90,16.28,540,2.78
...,...,...,...,...,...,...,...,...
6119,XS2284324667,2021-08-27,29.73,29.91,29.71,29.96,2335,1.71
6120,XS2314659447,2021-08-26,7.90,7.91,7.90,7.91,0,NaN
6121,XS2314659447,2021-08-27,7.92,7.98,7.92,7.98,0,0.94
6122,XS2314660700,2021-08-26,15.65,15.90,15.65,15.90,0,NaN
